# Лабораторна 4.1
## Завдання щодо генерації текстів або машинного перекладу (на вибір) на базі рекурентних мереж або трансформерів (на вибір). 
## Вирішіть завдання щодо генерації текстів або машинного перекладу. Особливо вітаються україномовні моделі.  

In [1]:
import keras
from keras import layers

import numpy as np
import random
import io

2023-12-18 20:34:48.726279: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path = '/Users/andrii.p/Desktop/Lisova_pisnya_1371650989.txt'

In [3]:
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))


Corpus length: 92883
Total chars: 60
Number of sequences: 30948


In [4]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)

In [6]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
model.fit(x, y, batch_size=128, epochs=20)

Epoch 1/20
242/242 [==============================] - 11s 39ms/step - loss: 2.6694
Epoch 2/20
242/242 [==============================] - 9s 39ms/step - loss: 2.2324
Epoch 3/20
242/242 [==============================] - 10s 41ms/step - loss: 2.0658
Epoch 4/20
242/242 [==============================] - 10s 40ms/step - loss: 1.9399
Epoch 5/20
242/242 [==============================] - 10s 40ms/step - loss: 1.8404
Epoch 6/20
242/242 [==============================] - 10s 40ms/step - loss: 1.7394
Epoch 7/20
242/242 [==============================] - 10s 41ms/step - loss: 1.6463
Epoch 8/20
242/242 [==============================] - 10s 40ms/step - loss: 1.5561
Epoch 9/20
242/242 [==============================] - 10s 41ms/step - loss: 1.4735
Epoch 10/20
242/242 [==============================] - 10s 41ms/step - loss: 1.3956
Epoch 11/20
242/242 [==============================] - 12s 52ms/step - loss: 1.3278
Epoch 12/20
242/242 [==============================] - 12s 48ms/step - loss: 1.2706
Ep

In [9]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = np.random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
generate_text(200,0.2)